In [ ]:
import gensim
import os
import collections
import smart_open
import random


import sys
sys.path.insert(0, '../models/')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""
from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
import keras
import tensorflow as tf
#from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
from keras.models import model_from_json
from keras.models import model_from_yaml
from sklearn.model_selection import train_test_split

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility

from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
import keras
from keras.initializers import RandomUniform
import keras.backend as K
from random import shuffle



#from  pyfasta import Fasta
import gzip
import numpy as np
import re
import random
#from data_gen import  DataGenerator_safe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model =gensim.models.word2vec.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/my_word2vec-model-DNA-3mer')

In [ ]:
model.corpus_count

In [ ]:
#model.most_similar('AAC')

In [ ]:
#model.wv['AAA']

In [ ]:
######## Read data ###############################
def read_seq_new(seq_file,label):
    seq_list = []
    seq = ''
    with open(seq_file, 'r') as fp:
        for line in fp:
            if line[0] == '>':
                if len(seq):
                    seq_list.append(seq)                    
                seq = ''
            else:
                seq = seq + line[:]
        
        if len(seq):
            seq_list.append(seq) 
    seq_data=[]
    all_data=[]
    for s in seq_list:
        tmp=s[0:81]
        seq_data.append(tmp)
        all_data.append([label])
    return seq_data, all_data


In [ ]:
seq_pos_data, all_data_pos = read_seq_new('/content/drive/MyDrive/Colab Notebooks/Bpromoter.fasta',1.0)
seq_neg_data, all_data_neg= read_seq_new('/content/drive/MyDrive/Colab Notebooks/Bnon_promoter.fasta',0.0)

seq_pos_data=np.asarray(seq_pos_data)
seq_neg_data=np.asarray(seq_neg_data)


In [ ]:
len(seq_neg_data)

In [ ]:
len(seq_pos_data)

In [ ]:
#all_data_pos[0]

In [ ]:
x_pos = np.zeros((len(seq_pos_data),81,100)) # 41 is lenght of sequence 41-2=39, 100 is set in main file length of sentence
for ix, seq in enumerate(seq_pos_data):
    for iy in range(78):
        vec = model.wv[seq[iy:iy+3]]
        x_pos[ix,iy,:]=vec

In [ ]:
x_pos.shape

In [ ]:
x_neg = np.zeros((len(seq_neg_data),81,100))
for ix, seq in enumerate(seq_neg_data):
    for iy in range(78):
        vec = model.wv[seq[iy:iy+3]]
        x_neg[ix,iy,:]=vec

In [ ]:
x_neg.shape

In [ ]:
y= len(x_pos)*[1]+len(x_neg)*[0]

In [ ]:
shuffled_indices =np.random.randint(0, len(y),len(y))
shuffled_indices

In [ ]:
x = np.concatenate((x_pos,x_neg))
len(x)

In [ ]:
y=np.asarray(y)

In [ ]:
y=y[shuffled_indices]

In [ ]:
y

In [ ]:
x=x[shuffled_indices]


In [ ]:
x_train,x_val,y_train,y_val= train_test_split(x,y, test_size=0.2)
x_train=np.asarray(x_train,dtype=np.float)
x_val=np.asarray(x_val,dtype=np.float)
y_train=np.asarray(y_train,dtype=np.float)
y_val=np.asarray(y_val,dtype=np.float)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
class CustomModelCheckpoint(keras.callbacks.Callback):
    def __init__(self, model, path):
        self.path = path
        self.model_for_saving = model
    def on_epoch_end(self, epoch, logs=None):
        loss = logs['val_loss']
        print("\nSaving model to : {}".format(self.path.format(epoch=epoch, val_loss=loss)))
        self.model_for_saving.save_weights(self.path.format(epoch=epoch, val_loss=loss), overwrite=True)
def step_decay(epoch):
    lrate=[0.001]*30+[0.0001]*170
    print(lrate[epoch])
    return lrate[epoch]

In [ ]:
model_name= 'shahid-word2vector'
result_folder= 'results-dataset-1-6mer/'
if not os.path.exists(result_folder+model_name):
    os.makedirs(result_folder+model_name)

model_results_folder=result_folder+model_name+ '/'
best_weights = model_results_folder + 'best_weights.h5'
last_weights = model_results_folder + 'last_weights.h5'
model_arch = model_results_folder + 'model.jsno'
history_pth = model_results_folder + 'results.pickle'

In [ ]:
!pip install keras-tcn
from tcn import TCN

TCN Model

In [ ]:
inp=Input((81,100))

x= TCN(32,13,padding='causal',use_skip_connections=True,dropout_rate=0.2,
           return_sequences=False,activation='relu',
             kernel_initializer='he_normal',
            use_layer_norm=False)(inp)
x=Dense(52, activation='relu')(x)

x=Dense(1, activation='sigmoid')(x) #output Layer
model= Model(inp,x)
print (model.summary())


In [ ]:
if not os.path.exists(result_folder+model_name):
    os.makedirs(result_folder+model_name)

model_results_folder=result_folder+model_name+ '/'
history_pth = model_results_folder + 'results.pickle'


model_json = model.to_json()
with open(model_results_folder+"model.json", "w") as json_file:
    json_file.write(model_json)

checkpoint = ModelCheckpoint(best_weights, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

checkpointsString = model_results_folder + 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
lrate = LearningRateScheduler(step_decay)
callbacks = [lrate, CustomModelCheckpoint(model, checkpointsString),checkpoint]



In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=0.001, amsgrad=True)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])


In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=40,shuffle=True, validation_split=0.1,callbacks=callbacks)

In [ ]:
import pickle
with open(history_pth, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


In [ ]:
#model.load_weights(best_weights)

In [ ]:
y_scores= model.predict(x_val)
y_true=y_val

In [ ]:
#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import jaccard_similarity_score, matthews_corrcoef
from sklearn.metrics import matthews_corrcoef

from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
%matplotlib inline
#Area under the ROC curve
fpr, tpr, thresholds = roc_curve((y_true), y_scores)
AUC_ROC = roc_auc_score(y_true, y_scores)
# test_integral = np.trapz(tpr,fpr) #trapz is numpy integration
print ("\nArea under the ROC curve: " +str(AUC_ROC))
roc_curve =plt.figure()
plt.plot(fpr,tpr,'-',label='Area Under the Curve (AUC = %0.4f)' % AUC_ROC)
plt.title('ROC curve')
plt.xlabel("FPR (False Positive Rate)")
plt.ylabel("TPR (True Positive Rate)")
plt.legend(loc="lower right")
#plt.savefig(path_experiment+"ROC.png")
plt.savefig('shahid-dataset-1-word2vec-ROC-4mer.png') ####################################################
#Precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
precision = np.fliplr([precision])[0]  #so the array is increasing (you won't get negative AUC)
recall = np.fliplr([recall])[0]  #so the array is increasing (you won't get negative AUC)
AUC_prec_rec = np.trapz(precision,recall)
print ("\nArea under Precision-Recall curve: " +str(AUC_prec_rec))
prec_rec_curve = plt.figure()
plt.plot(recall,precision,'-',label='Area Under the Curve (AUC = %0.4f)' % AUC_prec_rec)
plt.title('Precision - Recall curve')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower right")
#plt.savefig(path_experiment+"Precision_recall.png")

#Confusion matrix
threshold_confusion = 0.5
print ("\nConfusion matrix:  Custom threshold (for positive) of " +str(threshold_confusion))
y_pred = np.empty((y_scores.shape[0]))
for i in range(y_scores.shape[0]):
    if y_scores[i]>=threshold_confusion:
        y_pred[i]=1
    else:
        y_pred[i]=0
confusion = confusion_matrix(y_true, y_pred)
print (confusion)
accuracy = 0
if float(np.sum(confusion))!=0:
    accuracy = float(confusion[0,0]+confusion[1,1])/float(np.sum(confusion))
print ("Global Accuracy: " +str(accuracy))
specificity = 0
if float(confusion[0,0]+confusion[0,1])!=0:
    specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
print ("Specificity: " +str(specificity))
sensitivity = 0
if float(confusion[1,1]+confusion[1,0])!=0:
    sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
print ("Sensitivity: " +str(sensitivity))
precision = 0
if float(confusion[1,1]+confusion[0,1])!=0:
    precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
print ("Precision: " +str(precision))

#Jaccard similarity index
#jaccard_index = jaccard_similarity_score(y_true, y_pred, normalize=True)
#print ("\nJaccard similarity score: " +str(jaccard_index))

#F1 score
F1_score = f1_score(y_true, y_pred, labels=None, average='binary', sample_weight=None)
print ("\nF1 score (F-measure): " +str(F1_score))

mcc= matthews_corrcoef(y_true, y_pred)
print ('MCC:', mcc)



In [ ]:
def plot_confusion_matrix(cm, classes,
                              normalize=True,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        print(cm)

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        #plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()

In [ ]:
import itertools

In [ ]:
plot_confusion_matrix(confusion,classes=['F','T'])
plt.savefig('Shahid-dataset-1-confusionMatrix-4mer.jpg')